In [1]:
from keras.models import Model
from keras.layers import Dropout, Dense, Input
from keras.optimizers import Adam

import tensorflow as tf
import keras.backend as backend
import numpy as np

Using TensorFlow backend.


In [2]:
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True 
config.gpu_options.per_process_gpu_memory_fraction = 0.3 
sess = tf.compat.v1.Session(config=config)

In [3]:
LETTERS = 10

In [4]:
def create_model():
    inp = Input(shape=(10,))
    
    dense1 = Dense(215, activation='relu')(inp)
    drop1 = Dropout(0.02)(dense1)
    dense2 = Dense(215, activation='relu')(drop1)
    
    letters = Dense(LETTERS, activation='relu')(dense2)
    
    
    model = Model(inputs=inp, outputs=letters)
    model.compile(optimizer=Adam(lr=1e-4), loss='mae', metrics=['accuracy'])
    
    return model


In [5]:
model = create_model()

In [6]:
def conv_str_to_arr(text):
    text = text.upper()
    out = []
    for letter in text:        
        num = ord(letter)-64
        if 0 <= num <= 26:
            out.append(num)        
    return out

def conv_arr_to_str(arr):
    text = ''
    for num in arr:
        if 0 <= num <= 26:
            num = np.round(num)
            num = int(num)
            text += chr(num+64)
    return text

print(conv_str_to_arr('kotek'))
print(conv_arr_to_str([10, 14, 19, 4, 10]))

[11, 15, 20, 5, 11]
JNSDJ


In [7]:
def normalize_array(arr):
    if type(arr[0]) is not list:
        arr = [arr]
    
    arr = np.array([np.array(row) for row in arr])
    rows = len(arr)
    
    arr = arr / 26
    out = np.zeros((rows, LETTERS))
    for ri, row in enumerate(arr):
        out[ri, 0:len(row)] = row
    
    return out

arr = [conv_str_to_arr('kotkiem')]
out = normalize_array(arr)
print(out)

[[0.42307692 0.57692308 0.76923077 0.42307692 0.34615385 0.19230769
  0.5        0.         0.         0.        ]]


In [8]:
pairs = [('kotek', 'kot'), ('kotkiem', 'kot'), ('domkiem', 'dom'), ('autem', 'auto'), ('duda', 'duda'), ('kot', 'kot'),
        ('chomik', 'chomik'), ('chomikiem', 'chomik'), ('putina', 'putin')]
X = []
Y = []

for flex, normal in pairs:
    X.append(conv_str_to_arr(flex))
    Y.append(conv_str_to_arr(normal))


X = normalize_array(X)
Y = normalize_array(Y)
# print(X)
# print(Y)
    

In [9]:
history = model.fit(X, Y, validation_split=0.4, epochs=10000, verbose=0)

In [10]:
loss = history.history['loss']
accuracy = history.history['accuracy']

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

print(f"Accuracy: {accuracy[-1]:>2.4f}")
print(f"Loss: {loss[-1]:>2.4f}")

print(f"Test Accuracy: {val_accuracy[-1]:>2.4f}")
print(f"Test loss: {val_loss[-1]:>2.4f}")

Accuracy: 1.0000
Loss: 0.0146
Test Accuracy: 0.7500
Test loss: 0.1159


In [11]:
test = X[0, :].reshape(-1, LETTERS)
print(test.shape)
out = model.predict(test)

a = conv_arr_to_str(X[0, :]*26)
b = conv_arr_to_str(out[0]*26)

print(a)
print(b)


(1, 10)
KOTEK@@@@@
KOT@@@@@@@
